In [95]:
import numpy as np
import pandas as pd

In [96]:
def get_data(path):
    df = pd.read_csv(path, skiprows=72, header=None, sep=r'\s+', index_col=0)
    df.columns = [f'A{i}' for i in range(1, 16)] + ['B']
    df = df.reset_index(drop=True)
    return df.iloc[:, :-1], df.iloc[:, -1]

In [97]:
def dataLoader(X_train,y_train,batch_size=16):
    idx = np.arange(X_train.shape[0])
    np.random.shuffle(idx)
    X_train,y_train = X_train[idx],y_train[idx]
    for i in range(0,X_train.shape[0],batch_size):
        start,end = i,min(X_train.shape[0],i+batch_size)
        yield X_train[start:end],y_train[start:end]

In [98]:
def normalize_and_add_ones(X):
    X = np.array(X)
    X_max = np.max(X,axis=0)
    X_min = np.min(X,axis=0)
    X_normalized = (X-X_min) / (X_max-X_min)
    ones = np.ones(X_normalized.shape[0])
    return np.column_stack((ones,X_normalized))

In [99]:
class RidgeRegression:
    def __init__(self):
        pass
    def fit(self,X_train,y_train,LAMBDA):
        W = np.linalg.inv(X_train.T.dot(X_train) + LAMBDA*np.eye(X_train.shape[1])).dot(X_train.T.dot(y_train))
        return W
    def dataLoader(X_train,y_train,batch_size=16):
        idx = np.arange(X_train.shape[0])
        np.random.shuffle(idx)
        X_train,y_train = X_train[idx],y_train[idx]
        for i in range(0,X_train.shape[0],batch_size):
            start,end = i,min(X_train.shape[0],i+batch_size)
            yield X_train[start:end],y_train[start:end]

    def fit_gradient(self,X_train,y_train,LAMBDA,batch_size,lr,epoch=10):
        W = np.random.randn(X_train.shape[1])
        last_loss = 10e+8
        for _ in range(epoch):
            for x,y in dataLoader(X_train,y_train,batch_size):
                W -= lr *( X_train.T.dot(X_train.dot(W)-y_train) + LAMBDA * W)
            new_loss = self.computeRSS(self.predict(W,X_train),y_train)
            if np.abs(new_loss-last_loss)<=1e-5:
                break
            last_loss = new_loss
            if _%100==0:
                print(last_loss)
        return W
    
    
    def predict(self,W,X_new):
        return np.array(X_new).dot(W)

    def computeRSS(self,Y_new,Y_pred):
        return np.mean((Y_new-Y_pred)**2)

    def getTheBestLAMBDA(self,X_train,y_train):

        def crossValidation(num_folds,LAMBDA):
            row_ids = np.arange(X_train.shape[0])
            valid_ids = np.split(row_ids[:len(row_ids)-len(row_ids) % num_folds],num_folds)
            valid_ids[-1] = np.append(valid_ids[-1],row_ids[len(row_ids)-len(row_ids) % num_folds:])
            train_ids = [[k for k in row_ids if k not in valid_ids[i]] for i in range(num_folds)]
            avg_RSS = 0
            for i in range(num_folds):
                W = self.fit(X_train[train_ids[i]],y_train[train_ids[i]],LAMBDA)
                y_pred = self.predict(W,X_train[valid_ids[i]])
                avg_RSS+=self.computeRSS(y_train[valid_ids[i]],y_pred)
                return avg_RSS / num_folds

        def rangeScan(best_LAMBDA,min_RSS,LAMBDA_values):
            for current_LAMBDA in LAMBDA_values:
                avg_RSS = crossValidation(num_folds=2,LAMBDA=current_LAMBDA)
                if avg_RSS < min_RSS:
                    best_LAMBDA = current_LAMBDA
                    min_RSS = avg_RSS
            return best_LAMBDA,min_RSS

        best_LAMBDA, min_RSS = rangeScan(best_LAMBDA=0,min_RSS=1000**2,LAMBDA_values=range(50))

        LAMBDA_values = np.arange(max(0,(best_LAMBDA-1)*1000,(best_LAMBDA+1)*1000,1))*1.0/1000
        best_LAMBDA,min_RSS = rangeScan(best_LAMBDA=best_LAMBDA,min_RSS=min_RSS,LAMBDA_values=LAMBDA_values)
        
        return best_LAMBDA

In [100]:
X,y = get_data('Data/DeathRate.txt')

In [101]:
X = normalize_and_add_ones(X)

In [102]:
X_train,y_train = X[:50],y[:50]
X_test,y_test = X[50:],y[50:]

In [103]:
ridge_reg = RidgeRegression()
best_LAMBDA = ridge_reg.getTheBestLAMBDA(X_train,y_train)

In [104]:
best_LAMBDA

0.018

In [105]:
W_learned = ridge_reg.fit(X_train,y_train,best_LAMBDA)

In [106]:
y_pred = ridge_reg.predict(W_learned,X_test)

In [107]:
loss = ridge_reg.computeRSS(y_test,y_pred)

In [108]:
W_gradient = ridge_reg.fit_gradient(X_train,y_train,best_LAMBDA,batch_size=10,lr=0.01,epoch=1000)

374263.17878767184
1367.2416055789502
1013.8742222765867
884.7089142046051
830.9890875737989
805.8313660520763
792.729866806516
785.25145187456
780.6241370155641
777.5445870599875


In [109]:
loss_grad = ridge_reg.computeRSS(ridge_reg.predict(W_gradient,X_test),y_pred)

In [110]:
print(f'Loss with formula: {loss:.2f}')
print(f'Loss with gradient: {loss_grad:.2f}')

Loss with formula: 1416.56
Loss with gradient: 0.55
